In [ ]:
import pandas as pd
import numpy as np
from os import path, makedirs

In [ ]:
base_path = path.join('data_output', 'urbanismo')
if not path.exists(base_path):
    makedirs(base_path)
def _normalize_headers(cols):
    return [str(c).strip() for c in cols]
def _clean_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = _normalize_headers(df.columns)
    for c in df.columns:
        if df[c].dtype == object:
            df[c] = df[c].astype(str).str.strip().replace({"": np.nan})
    return df

In [ ]:
df_atd = pd.read_csv("dados/urbanismo/atendimentos_viagens.csv",
                     sep=";", dtype=str)

df_loa = pd.read_csv("dados/urbanismo/loa_atende.csv",
                     sep=";", dtype=str)

df_km = pd.read_csv("dados/urbanismo/quilometragemKM.csv",
                     sep=";", dtype=str)

In [ ]:
# Ajustar para formato iso data ex: AAAA-MM-DD
df_atd['data'] = pd.to_datetime(df_atd['data']).dt.strftime('%Y-%m-%d')
df_atd = _clean_df(df_atd)
df_atd

In [ ]:
df_loa

In [ ]:
# Aqui eu despivoto
df_km_melted = pd.melt(
    df_km,
    id_vars=['Empresa', 'Lote', 'Mês'],
    value_vars=['Valor_Quilometragem_2022', 'Valor_Quilometragem_2023', 'Valor_Quilometragem_2024'],
    var_name='Ano',
    value_name='Valor_Quilometragem'
)

# Ajsuto a col "Ano" para ano
df_km_melted['Ano'] = df_km_melted['Ano'].str.extract(r'(\d{4})')

# Criei a coluna data com a primeira data de cada mês
df_km_melted['Data'] = pd.to_datetime(df_km_melted['Ano'] + '-' + df_km_melted['Mês'] + '-01', format='%Y-%B-%d')

df_km_melted

In [ ]:
base_path = path.join('data_output', 'urbanismo')

if not path.exists(base_path):
    makedirs(base_path)

for name, df in [
                 ('atendimentos_viagens', df_atd),
                 ('loa_atende', df_loa),
                 ('quilometragemKM', df_km_melted)
                 ]:
    filepath = path.join(base_path, f'{name}.csv')

    df.to_csv(filepath,
              index=False,
              sep=';',
              decimal=',',
              encoding='latin1')